In [1]:
import numpy as np
import pandas as pd

In [2]:
usfa = pd.read_csv("/home/jovyan/work/usfa-registry-CA.csv")
usfa.head()

,FDID,Fire dept name,HQ addr1,HQ addr2,HQ city,HQ state,HQ zip,Mail addr1,Mail addr2,Mail PO box,...,Dept Type,Organization Type,Website,Number Of Stations,Active Firefighters - Career,Active Firefighters - Volunteer,Active Firefighters - Paid per Call,Non-Firefighting - Civilian,Non-Firefighting - Volunteer,Primary agency for emergency mgmt
0,31800,"233rd Firefighting Team, CA Army National Guard",850 All America City BLVD,NaN,Roseville,CA,95678-1594,850 All America City BLVD,NaN,NaN,...,Mostly volunteer,NaN,NaN,1,3,0,18,0,0,No
1,25005,Adin Fire Protection District,108 Center ST,NaN,Adin,CA,96006,NaN,NaN,PO Box 16,...,Volunteer,NaN,https://adinfire.org/,1,0,16,0,0,0,Yes
2,34003,Aerojet Fire Services,HWY 50 and Hazel AVE,PO Box 13222,Sacramento,CA,95813-6000,HWY 50 and Hazel AVE,NaN,PO Box 13222,...,Career,NaN,NaN,1,15,0,0,0,16,NaN
3,NaN,Air Force Plant 42 Fire Department,2501 E Avenue P,NaN,Palmdale,CA,93551,2501 E Avenue P,NaN,PO Box,...,Career,NaN,NaN,2,52,0,0,0,0,NaN
4,01008,Alameda County Fire Department,6363 Clark AVE,NaN,Dublin,CA,94568-3001,6363 Clark AVE,NaN,NaN,...,Mostly career,NaN,http://www.acgov.org/fire/,29,350,50,0,92,0,Yes


Let's convert address information to longitude/ latitude information using geopandas and geopy, and then map each to a county.

In [3]:
# make address column (concatenate HQaddr1 , HQ addr2 , HQ city, HQ state, HQ zip)
usfa['HQ addr1'] = usfa['HQ addr1'].replace(np.nan, '', regex=True)
usfa['HQ addr2'] = usfa['HQ addr2'].replace(np.nan, '', regex=True)
usfa['HQ city'] = usfa['HQ city'].replace(np.nan, '', regex=True)
usfa['HQ state'] = usfa['HQ state'].replace(np.nan, '', regex=True)
usfa['HQ zip'] = usfa['HQ zip'].replace(np.nan, '', regex=True)
usfa['address']= usfa['HQ addr1']+", "+ usfa['HQ addr2'] + ", " + usfa['HQ city'] + ", " + usfa['HQ state'] + ", " + usfa['HQ zip']

usfa.head()

,FDID,Fire dept name,HQ addr1,HQ addr2,HQ city,HQ state,HQ zip,Mail addr1,Mail addr2,Mail PO box,...,Organization Type,Website,Number Of Stations,Active Firefighters - Career,Active Firefighters - Volunteer,Active Firefighters - Paid per Call,Non-Firefighting - Civilian,Non-Firefighting - Volunteer,Primary agency for emergency mgmt,address
0,31800,"233rd Firefighting Team, CA Army National Guard",850 All America City BLVD,,Roseville,CA,95678-1594,850 All America City BLVD,NaN,NaN,...,NaN,NaN,1,3,0,18,0,0,No,"850 All America City BLVD, , Roseville, CA, 95..."
1,25005,Adin Fire Protection District,108 Center ST,,Adin,CA,96006,NaN,NaN,PO Box 16,...,NaN,https://adinfire.org/,1,0,16,0,0,0,Yes,"108 Center ST, , Adin, CA, 96006"
2,34003,Aerojet Fire Services,HWY 50 and Hazel AVE,PO Box 13222,Sacramento,CA,95813-6000,HWY 50 and Hazel AVE,NaN,PO Box 13222,...,NaN,NaN,1,15,0,0,0,16,NaN,"HWY 50 and Hazel AVE, PO Box 13222, Sacramento..."
3,NaN,Air Force Plant 42 Fire Department,2501 E Avenue P,,Palmdale,CA,93551,2501 E Avenue P,NaN,PO Box,...,NaN,NaN,2,52,0,0,0,0,NaN,"2501 E Avenue P, , Palmdale, CA, 93551"
4,01008,Alameda County Fire Department,6363 Clark AVE,,Dublin,CA,94568-3001,6363 Clark AVE,NaN,NaN,...,NaN,http://www.acgov.org/fire/,29,350,50,0,92,0,Yes,"6363 Clark AVE, , Dublin, CA, 94568-3001"


In [4]:
!pip install geopandas
!pip install geopy

     |████████████████████████████████| 962 kB 17.6 MB/s 
     |████████████████████████████████| 6.4 MB 80.6 MB/s 
     |████████████████████████████████| 14.8 MB 74.1 MB/s 
     |████████████████████████████████| 1.0 MB 57.0 MB/s 
     |████████████████████████████████| 82 kB 2.7 MB/s 
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 111 kB 25.1 MB/s 
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.


In [5]:
from geopy.extra.rate_limiter import RateLimiter
from geopy. geocoders import Nominatim


In [6]:
# CA_county_fips = {1: "ALAMEDA", 21: "MARIN", 41: "SAN MATEO", 
# 2: "ALPINE", 22: "MARIPOSA", 42: "SANTA BARBARA",
# 03: "AMADOR",  23: "MENDOCINO" , 43: "SANTA CLARA",
# 04: "BUTTE", 24: "MERCED", 44: "SANTA CRUZ",
# 05: "CALAVERAS", 25: "MODOC", 45: "SHASTA",
# 06: "COLUSA", 26: "MONO", 46: "SIERRA", 
# 07: "CONTRA COSTA" 27 MONTEREY 47 SISKIYOU 
# 08: "DEL NORTE" 28 NAPA 48 SOLANO
# 09: "EL DORADO" 29 49 SONOMA
# 10: "FRESNO" 30 ORANGE 50 STANISLAUS
# 11: "GLENN" 31 PLACER 51 SUTTER
# 12: "HUMBOLDT" 32 PLUMAS 52 TEHAMA
# 13: "IMPERIAL" 33 RIVERSIDE 53 TRINITY
# 14: "INYO" 34 SACRAMENTO 54 TULARE
# 15: "KERN" 35 SAN BENITO 55 TUOLUMNE
# 16: "KINGS" 36 SAN BERNARDINO 56 VENTURA
# 17: LAKE 37 SAN DIEGO 57 YOLO
# 18: LASSEN 38 SAN FRANCISCO 58 YUBA
# 19: LOS ANGELES 39 SAN JOAQUIN
# 20: MADERA 40 SAN LUIS OBISPO}



In [7]:
usfa.shape
usfa = usfa.drop(['HQ addr1', 'HQ addr2', 'HQ city', 'HQ state', 'Mail addr1', 'Mail addr2', 'Mail PO box', 'Mail city', 'Mail state', 'Mail zip', 'HQ phone', 'HQ fax'], axis = 1)
usfa.head()


,FDID,Fire dept name,HQ zip,County,Dept Type,Organization Type,Website,Number Of Stations,Active Firefighters - Career,Active Firefighters - Volunteer,Active Firefighters - Paid per Call,Non-Firefighting - Civilian,Non-Firefighting - Volunteer,Primary agency for emergency mgmt,address
0,31800,"233rd Firefighting Team, CA Army National Guard",95678-1594,61,Mostly volunteer,NaN,NaN,1,3,0,18,0,0,No,"850 All America City BLVD, , Roseville, CA, 95..."
1,25005,Adin Fire Protection District,96006,49,Volunteer,NaN,https://adinfire.org/,1,0,16,0,0,0,Yes,"108 Center ST, , Adin, CA, 96006"
2,34003,Aerojet Fire Services,95813-6000,67,Career,NaN,NaN,1,15,0,0,0,16,NaN,"HWY 50 and Hazel AVE, PO Box 13222, Sacramento..."
3,NaN,Air Force Plant 42 Fire Department,93551,37,Career,NaN,NaN,2,52,0,0,0,0,NaN,"2501 E Avenue P, , Palmdale, CA, 93551"
4,01008,Alameda County Fire Department,94568-3001,1,Mostly career,NaN,http://www.acgov.org/fire/,29,350,50,0,92,0,Yes,"6363 Clark AVE, , Dublin, CA, 94568-3001"


In [8]:

# locator = Nominatim(user_agent= 'myGeocoder')
# geocode = RateLimiter(locator.geocode, min_delay_seconds = 0.5)
# usfa['location'] = usfa['address'].apply(geocode)
# usfa['point'] = usfa['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# usfa[['latitude', 'longitude', 'altitude']] = pd.DataFrame(usfa['point'].tolist(), index = usfa.index)

In [9]:
usfa.columns

Index(['FDID', 'Fire dept name', 'HQ zip', 'County', 'Dept Type',
       'Organization Type', 'Website', 'Number Of Stations',
       'Active Firefighters - Career', 'Active Firefighters - Volunteer',
       'Active Firefighters - Paid per Call', 'Non-Firefighting - Civilian',
       'Non-Firefighting - Volunteer', 'Primary agency for emergency mgmt',
       'address'],
      dtype='object')

In [10]:

# next: add in county column -- usfa_with_counties

In [11]:
# Use zips to county crosswalk to pull in county
zips = pd.read_csv("ZIP-COUNTY-FIPS_2017-06.csv")
# https://www.kaggle.com/danofer/zipcodes-county-fips-crosswalk

In [12]:
usfa['5_dig_zip'] = usfa['HQ zip'].str[:5].astype(str)
zips['ZIP'] = zips['ZIP'].astype(str)

# Merge on zip to get county
usfa_with_counties = pd.merge(usfa, zips,  how='left', left_on=['5_dig_zip'], right_on = ['ZIP'])


In [13]:
usfa_with_counties.head()
# Checking for NAs after merge - none
usfa_with_counties[usfa_with_counties['COUNTYNAME'].isna()]

,FDID,Fire dept name,HQ zip,County,Dept Type,Organization Type,Website,Number Of Stations,Active Firefighters - Career,Active Firefighters - Volunteer,...,Non-Firefighting - Civilian,Non-Firefighting - Volunteer,Primary agency for emergency mgmt,address,5_dig_zip,ZIP,COUNTYNAME,STATE,STCOUNTYFP,CLASSFP


In [14]:
# Clean county names
banned = ['County']
f = lambda x: ' '.join([item for item in x.split() if item not in banned])
usfa_with_counties["COUNTYNAME"] = usfa_with_counties["COUNTYNAME"].apply(f)

In [15]:
usfa_with_counties.head()

,FDID,Fire dept name,HQ zip,County,Dept Type,Organization Type,Website,Number Of Stations,Active Firefighters - Career,Active Firefighters - Volunteer,...,Non-Firefighting - Civilian,Non-Firefighting - Volunteer,Primary agency for emergency mgmt,address,5_dig_zip,ZIP,COUNTYNAME,STATE,STCOUNTYFP,CLASSFP
0,31800,"233rd Firefighting Team, CA Army National Guard",95678-1594,61,Mostly volunteer,NaN,NaN,1,3,0,...,0,0,No,"850 All America City BLVD, , Roseville, CA, 95...",95678,95678,Placer,CA,6061,H1
1,31800,"233rd Firefighting Team, CA Army National Guard",95678-1594,61,Mostly volunteer,NaN,NaN,1,3,0,...,0,0,No,"850 All America City BLVD, , Roseville, CA, 95...",95678,95678,Sacramento,CA,6067,H1
2,25005,Adin Fire Protection District,96006,49,Volunteer,NaN,https://adinfire.org/,1,0,16,...,0,0,Yes,"108 Center ST, , Adin, CA, 96006",96006,96006,Modoc,CA,6049,H1
3,34003,Aerojet Fire Services,95813-6000,67,Career,NaN,NaN,1,15,0,...,0,16,NaN,"HWY 50 and Hazel AVE, PO Box 13222, Sacramento...",95813,95813,Sacramento,CA,6067,H1
4,NaN,Air Force Plant 42 Fire Department,93551,37,Career,NaN,NaN,2,52,0,...,0,0,NaN,"2501 E Avenue P, , Palmdale, CA, 93551",93551,93551,Los Angeles,CA,6037,H1


## Explore Fire Camps

In [16]:
# incarcerated firefighter stats- "Fire camps". 
# Each row is a name of a fire/conservation camp. 2020_staff_num is the number of staff members, then the number of inmates, the the total capacity
fire_camps_df = pd.read_csv("/home/jovyan/work/CA_conservation_camps_2020.csv")
fire_camps_df['Zip'] = fire_camps_df['Zip'].astype(str)
fire_camps_df.head()


,Name,Zip,Address,2020_staff_num,2020_num_inmates,inmate_capacity
0,Alder #20,95548,P.O. Box 906 Klamath Ca. 95548,9,39,100
1,Acton #11,93510,"8800 Soledad Canyon Road, Acton, CA 93510",8,45,80
2,Antelope #25,96127,"P.O. Box 270160, Susanville, CA 96127",10,50,120
3,Baseline #30,95327,"16809 Peoria Flat Road, Jamestown, CA, 95327",10,56,120
4,Bautista #36,92544,"33015 Bautista Rd, Hemet, CA",10,45,120


In [17]:
fire_camps_df = pd.merge(fire_camps_df, zips,  how='left', left_on=['Zip'], right_on = ['ZIP'])
fire_camps_df = fire_camps_df.drop(['Zip'], axis = 1)
banned = ['County']
f = lambda x: ' '.join([item for item in x.split() if item not in banned])
fire_camps_df["COUNTYNAME"] = fire_camps_df["COUNTYNAME"].apply(f)
fire_camps_df.head()


,Name,Address,2020_staff_num,2020_num_inmates,inmate_capacity,ZIP,COUNTYNAME,STATE,STCOUNTYFP,CLASSFP
0,Alder #20,P.O. Box 906 Klamath Ca. 95548,9,39,100,95548,Del Norte,CA,6015,H1
1,Acton #11,"8800 Soledad Canyon Road, Acton, CA 93510",8,45,80,93510,Los Angeles,CA,6037,H1
2,Antelope #25,"P.O. Box 270160, Susanville, CA 96127",10,50,120,96127,Lassen,CA,6035,H1
3,Baseline #30,"16809 Peoria Flat Road, Jamestown, CA, 95327",10,56,120,95327,Tuolumne,CA,6109,H1
4,Bautista #36,"33015 Bautista Rd, Hemet, CA",10,45,120,92544,Riverside,CA,6065,H1


Let's explore the 2020 incarcerated firefighter population and the usage of prison labor to fight fires. Lets also compare this to the normal firefighting population.


In [18]:
print("The total capacity for inmate firefighters in CA: ", fire_camps_df.inmate_capacity.sum())
print("The number of inmate firefighters in 2020 in CA (lower than usual due to COVID): " , fire_camps_df['2020_num_inmates'].sum())

The total capacity for inmate firefighters in CA:  5349
The number of inmate firefighters in 2020 in CA (lower than usual due to COVID):  2230


In [19]:
print("The total number of active Career firefighters in CA (non-inmate):", usfa_with_counties['Active Firefighters - Career'].sum())
print("The total number of active Volunteer firefighters in CA (non-inmate):", usfa_with_counties['Active Firefighters - Volunteer'].sum())
print("The total number of active Paid per Call firefighters in CA (non-inmate):", usfa_with_counties['Active Firefighters - Paid per Call'].sum())

The total number of active Career firefighters in CA (non-inmate): 47242
The total number of active Volunteer firefighters in CA (non-inmate): 11406
The total number of active Paid per Call firefighters in CA (non-inmate): 6271


One little note: we are unsure if each firefighter's membership to a specific firestation is exclusive- i.e. can a volunteer firefighter belong to multiple stations? are the numbers above thus overestimates? 

In [20]:
# ----------------------------- TODO: Here's the data we need to fill in! --------------------------------------
"""county_list = ['Alameda', 'Alpine', 'Amador',...]
county_inmate_list = [] # list to hold X in formula above
county_non_inmate_list = [] # list to hold Y in formula above
county_fire_cost_list = [] # list to hold 'fire cost' (per month) in formula above
monthly_inmate_cost =  # c_1 in the formula above
monthly_non_inmate_cost =  # c_2 in the formula above
monthly_cost_saved_per_firefighter =  # c_3 in the formula above
"""

# ------------------- make table -------------------
# | county |  num_inmate_firefighters |  num_career_ff | num_volunteer_ff | num_paid_percall_ff |




"county_list = ['Alameda', 'Alpine', 'Amador',...]\ncounty_inmate_list = [] # list to hold X in formula above\ncounty_non_inmate_list = [] # list to hold Y in formula above\ncounty_fire_cost_list = [] # list to hold 'fire cost' (per month) in formula above\nmonthly_inmate_cost =  # c_1 in the formula above\nmonthly_non_inmate_cost =  # c_2 in the formula above\nmonthly_cost_saved_per_firefighter =  # c_3 in the formula above\n"

In [21]:
banned = ['County']
f = lambda x: ' '.join([item for item in x.split() if item not in banned])
zips["COUNTYNAME"] = zips["COUNTYNAME"].apply(f)
print(len(zips[zips.STATE == "CA"].COUNTYNAME.unique()))


58


In [22]:
ff_counts_noninmate = usfa_with_counties[['COUNTYNAME', 'Active Firefighters - Career', 'Active Firefighters - Volunteer',
       'Active Firefighters - Paid per Call']]

ff_counts_noninmate = ff_counts_noninmate.groupby("COUNTYNAME").sum().reset_index()

inmate_counts = fire_camps_df[['COUNTYNAME', '2020_num_inmates', 'inmate_capacity']]
inmate_counts = inmate_counts.groupby("COUNTYNAME").sum().reset_index()

ff_counts = pd.merge(ff_counts_noninmate, inmate_counts, how = 'left', on = "COUNTYNAME")
ff_counts = ff_counts.fillna(0)
ff_counts.head()

ff_counts.to_csv("ff_counts.csv", index = False)
